In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import os
import pandas as pd
data = []

ext = ('.jpeg', 'jpg')

thisdir = '/content/drive/MyDrive/zay'

for r, d, f in os.walk(thisdir):
    for file in f:
      # if file.endswith(ext):
      data.append((r.split('/')[-1],os.path.join(r,file)))

df = pd.DataFrame(data,columns=['folder','file'])

In [ ]:
df

,folder,file
0,musangsulawesi,/content/drive/MyDrive/zay/musangsulawesi/0000...
1,musangsulawesi,/content/drive/MyDrive/zay/musangsulawesi/0000...
2,musangsulawesi,/content/drive/MyDrive/zay/musangsulawesi/0000...
3,musangsulawesi,/content/drive/MyDrive/zay/musangsulawesi/0000...
4,musangsulawesi,/content/drive/MyDrive/zay/musangsulawesi/0000...
...,...,...
2251,babirusasulawesi,/content/drive/MyDrive/zay/babirusasulawesi/00...
2252,babirusasulawesi,/content/drive/MyDrive/zay/babirusasulawesi/00...
2253,babirusasulawesi,/content/drive/MyDrive/zay/babirusasulawesi/00...
2254,babirusasulawesi,/content/drive/MyDrive/zay/babirusasulawesi/00...


In [ ]:
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import img_to_array

data_array = []

# Hyperparameters
HEIGHT = 150 # Image height
WIDTH = 150 # Image width
CHANNELS = 3 # Image channels

for gambar in df['file']:
  image = load_img(gambar, target_size=(HEIGHT, WIDTH, CHANNELS))
  image = img_to_array(image)
  data_array.append(image)

In [ ]:
df['data_array'] = data_array

In [ ]:
df.head()

,folder,file,data_array
0,musangsulawesi,/content/drive/MyDrive/zay/musangsulawesi/0000...,"[[[89.0, 80.0, 65.0], [146.0, 129.0, 119.0], [..."
1,musangsulawesi,/content/drive/MyDrive/zay/musangsulawesi/0000...,"[[[7.0, 7.0, 9.0], [9.0, 8.0, 13.0], [10.0, 11..."
2,musangsulawesi,/content/drive/MyDrive/zay/musangsulawesi/0000...,"[[[50.0, 77.0, 32.0], [61.0, 92.0, 35.0], [54...."
3,musangsulawesi,/content/drive/MyDrive/zay/musangsulawesi/0000...,"[[[19.0, 29.0, 5.0], [97.0, 107.0, 82.0], [78...."
4,musangsulawesi,/content/drive/MyDrive/zay/musangsulawesi/0000...,"[[[204.0, 205.0, 209.0], [199.0, 200.0, 204.0]..."


In [ ]:
# df.to_csv('Data zai.csv')

In [ ]:
# df['folder'].value_counts()

In [ ]:
# n = 138
# msk = df.groupby('folder')['folder'].transform('size') >= n
# df = pd.concat((df[msk].groupby('folder').sample(n=n), df[~msk]), ignore_index=True)

In [ ]:
# df['folder'].value_counts()

In [ ]:
import numpy as np

image_data = np.array(list(df['data_array']))
labels = np.array(df['folder'].values)

In [ ]:
image_data.shape

(2256, 150, 150, 3)

In [ ]:
from sklearn.preprocessing import OneHotEncoder

labelencoder_y = OneHotEncoder(sparse=False)
labels = labels.reshape(-1,1)
labels = labelencoder_y.fit_transform(labels)

In [ ]:
labels

array([[0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       ...,
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.]])

In [ ]:
# idx = np.arange(image_data.shape[0])
# np.random.shuffle(idx)
# image_data = image_data[idx]
# labels = labels[idx]

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(image_data, labels, test_size = 0.2, random_state = 101)

In [ ]:
print(f'SHAPE OF TRAINING IMAGE DATA : {x_train.shape}')
print(f'SHAPE OF TESTING IMAGE DATA : {x_test.shape}')
print(f'SHAPE OF TRAINING LABELS : {y_train.shape}')
print(f'SHAPE OF TESTING LABELS : {y_test.shape}')

SHAPE OF TRAINING IMAGE DATA : (1804, 150, 150, 3)
SHAPE OF TESTING IMAGE DATA : (452, 150, 150, 3)
SHAPE OF TRAINING LABELS : (1804, 10)
SHAPE OF TESTING LABELS : (452, 10)


In [ ]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255,
    rotation_range=40,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest')
val_datagen = ImageDataGenerator(rescale = 1/255.)

train_generator = train_datagen.flow(x_train, y_train, batch_size = 128, shuffle = True)
val_generator = val_datagen.flow(x_test, y_test, batch_size = 128, shuffle = True)

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dropout, Flatten, Dense
from tensorflow.keras.applications import VGG16

input_layer = Input(shape=(150, 150, 3))
hidden_layer_1 = VGG16(include_top=False,
    weights='imagenet')(input_layer)
hidden_layer_2 = Flatten()(hidden_layer_1)
hidden_layer_3 = Dense(256, activation='relu')(hidden_layer_2)
hidden_layer_4 = Dropout(0.2)(hidden_layer_3)
output_layer = Dense(10, activation='softmax')(hidden_layer_4)

model = Model(inputs=input_layer, outputs=output_layer)

In [ ]:
# for layer in model.layers:
#   layer.trainable = True
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 vgg16 (Functional)          (None, None, None, 512)   14714688  
                                                                 
 flatten (Flatten)           (None, 8192)              0         
                                                                 
 dense (Dense)               (None, 256)               2097408   
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 10)                2570      
                                                                 
Total params: 16,814,666
Trainable params: 16,814,666
Non-tra

In [ ]:
from tensorflow.keras.optimizers import Adam
model.compile(loss='categorical_crossentropy', #karena lebih dari 2 kelas, menggunakan categorical_crossentropy
              optimizer=Adam(learning_rate=1.0000e-04),
              metrics=['accuracy'])

In [ ]:
#callbacks
from tensorflow.keras.callbacks import Callback
class myCallback(Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy')>0.90 and logs.get('val_accuracy')>0.90):
      print("\nAkurasi telah sampai lebih dari 90%!")
      self.model.stop_training = True
callbacks = myCallback()

In [ ]:
model.fit(train_generator, 
          callbacks=[callbacks],
          validation_data=val_generator,
          epochs = 150, 
          batch_size = 64)

Epoch 1/150
15/15 [==============================] - 36s 1s/step - loss: 2.2583 - accuracy: 0.1735 - val_loss: 1.9765 - val_accuracy: 0.2677
Epoch 2/150
15/15 [==============================] - 14s 915ms/step - loss: 1.9467 - accuracy: 0.2932 - val_loss: 1.7714 - val_accuracy: 0.3606
Epoch 3/150
15/15 [==============================] - 14s 906ms/step - loss: 1.7128 - accuracy: 0.3747 - val_loss: 1.7022 - val_accuracy: 0.4093
Epoch 4/150
15/15 [==============================] - 14s 897ms/step - loss: 1.5536 - accuracy: 0.4379 - val_loss: 1.4751 - val_accuracy: 0.4602
Epoch 5/150
15/15 [==============================] - 13s 859ms/step - loss: 1.3497 - accuracy: 0.5105 - val_loss: 1.2108 - val_accuracy: 0.5752
Epoch 6/150
15/15 [==============================] - 14s 897ms/step - loss: 1.2778 - accuracy: 0.5399 - val_loss: 1.1961 - val_accuracy: 0.5863
Epoch 7/150
15/15 [==============================] - 14s 900ms/step - loss: 1.1901 - accuracy: 0.5576 - val_loss: 1.1191 - val_accuracy: 0.

In [ ]:
model.save('my_model.h5')

In [ ]:
model.save_weights('/content/drive/MyDrive/zay_backup/my_model_weight.h5')

In [ ]:
model.metrics_names

['loss', 'accuracy']

In [ ]:
import matplotlib.pyplot as plt

#plot accuracy
plt.plot(model.history.history['accuracy'], label='Training Accuracy')
plt.plot(model.history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.legend(['Train', 'Validation'], loc='lower right')
plt.show()

KeyError: ignored

In [ ]:
score = model.evaluate(val_generator, verbose=0)
print(score[1]*100)

71.46017551422119


In [ ]:
import pickle

filehandler = open("/content/drive/MyDrive/zay_backup/le.pkl","wb")
pickle.dump(labelencoder_y,filehandler)
filehandler.close()

Pengujian

In [ ]:
path = '/content/Babirusa.jpg'

def process_gambar(image):
  arr = []
  image = load_img(image, target_size=(150, 150, 3))
  image = img_to_array(image)
  arr.append(image)
  return arr

data_prediksi = process_gambar(path)

In [ ]:
data_prediksi = np.array(data_prediksi)
prediksi_gambar = model.predict(data_prediksi)

1/1 [==============================] - 0s 16ms/step


In [ ]:
print(labelencoder_y.inverse_transform(prediksi_gambar))

[['babirusasulawesi']]
